In [ ]:
import numpy as np
import pandas as pd
import polarDensity_helper as pc
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
plt.rcParams['axes.grid'] = False 

In [ ]:
'''
memb (str) should be synps or oocyt
ddg (bool) determines if ddg analysis called
lipids (str) points to a file to check lipids
enrich (bool) tells script to check enrichment
'''

# initialization
density = None
chol_bind = None
w3_bind = None
rat = None
chains_groups = ["POPE", "POPG"]
lipids = chains_groups
col = []
#chains_up, chains_lo = pc.prot_coord()
enrich = True

# get files to use
root = Path("..")
file_list = []
for lip in lipids:
    toadd = list(root.glob(f"{lip}*.dat*.dat") )
    file_list = np.append(file_list,toadd)

leaflets = ['low', 'upp']

In [ ]:
enrichments = pd.DataFrame(index=chains_groups, columns=leaflets)
counts = pd.DataFrame(index=chains_groups, columns=leaflets)

idx = 0
for fl in file_list:
    if idx == 0:
        rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(fl)

    filename = fl.name

    tmp_chain = filename.split('.')[0]
    tmp_nm = filename.split('.')[2]

    # This is a hack. The above part does not have a "flexible"
    # method to consider sim type (a, b ...)
    idx+=1
    toadd = np.loadtxt(fl, skiprows=1)
    toadd = toadd[:,3:-1]
    counts.at[tmp_chain,tmp_nm] = toadd
    enrichments.at[tmp_chain,tmp_nm] = pc._analysis_call_(fl, radius, dr, dth, frames, enrich=enrich)


In [ ]:
# Optional helix locations
try:
    helices_lwr = np.loadtxt(root.joinpath("Protein_coords_lwr.dat"))
    helices_upr = np.loadtxt(root.joinpath("Protein_coords_upr.dat"))
except FileNotFoundError:
    helices_lwr = None
    helices_upr = None
    print("Protein coordinates not found")

In [ ]:
def polar_plot(data_in, theta, radius, chains_groups, helices_lwr=None, helices_upr=None, vmax=2, vmid=1, vmin=0, colorbychain=True):
	# plots densities
	# data_in = array/list of density data
	# theta, radius = arrays of position bins
	# chains_groups = old name, really lipids to plot

    data_in = pc.sum_reps(data_in)
    fig = plt.figure(figsize=(10,10))
    gs1=gridspec.GridSpec(len(chains_groups),2,wspace=.15, hspace=0.15)
    plt.rcParams.update({'font.size': 10})
    norm1 = pc.MidpointNormalize(midpoint=vmid,vmin=vmin,vmax=vmax)
    cmap = plt.cm.RdBu_r#PuOr
    cmap.set_bad(color='black')
    grid = 0
    #chains_up, chains_lo = prot_coord()
    #sub = ["g",'m','grey','green','cyan']#original

    #  orange   light blue   green      amber      blue       red       purple 
    #"#E69F00"  "#56B4E9"  "#009E73"  "#F5C710"  "#0072B2"  "#D55E00"  "#CC79A7" 
    sub = ["#E69F00", "#56B4E9", "#009E73", "#F5C710", "#0072B2", "#D55E00", "#CC79A7" ]

    for cg in chains_groups:
        for leaf in data_in.columns:
            ax = plt.subplot(gs1[grid],projection="polar")
            ax.grid(False)
            toplot = data_in.at[cg,leaf]
            s = ax.pcolormesh(theta, 
                              radius, 
                              toplot,
                              cmap=cmap,
                              norm=norm1,
                              zorder=0,
                              edgecolors='none',
                              linewidth=0,
                              )
            s.set_edgecolor('face')
            if grid%2==0:
                ax.set_ylabel(cg)
            if grid < 2:
                ax.set_title(leaf)
                
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            grid = grid + 1
            
            if leaf=="Outer":
                helices = helices_upr
            else:
                helices = helices_lwr

            if helices is not None:
                if len(np.shape(helices))==1:
                    helices = np.reshape(helices, (1,len(helices)))
                for i,pro in enumerate(helices[:]):
                    if colorbychain:
                        colors = sub[i]
                    else:
                        colors = sub[:len(pro[::2])]
                    ax.scatter(np.deg2rad(pro[1::2]),
                                pro[::2],
                                color=colors,
                                linewidth=6,
                                zorder=1, 
                                s=np.shape(data_in)[0]*10,
                                )


    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.21, .89, 0.5, 0.008])
    fig.colorbar(s, cax=cbar_ax,ticks=np.linspace(vmin,vmax,5),orientation="horizontal")

    #plt.tight_layout()
    return fig, ax

In [ ]:
radius = radius-5

In [ ]:
fig, ax = polar_plot(enrichments, theta, radius, chains_groups, helices_lwr, helices_upr, colorbychain=False)
plt.savefig("old_enrichment.pdf")
plt.show()

In [ ]:
# A = radius*dr*dth
# densities = counts.copy()
# for lip in lipids:
#     for leaf in leaflets:
#         densities.at[lip, leaf] = counts.loc[lip, leaf]/A

# fig, ax = polar_plot(densities, theta, radius, chains_groups, helices_lwr, helices_upr, vmax=0.2, vmid=0.1, vmin=0)
# plt.savefig("densities.pdf")
# plt.show()

In [ ]:
# freqs, edges = np.histogram(np.reshape(densities.iloc[0,0],np.product(np.shape(densities.iloc[0,0]))), density=True)
# centers = edges[0:-1]+(edges[0]+edges[1])/2
# fig, ax = plt.subplots()
# ax.plot(centers, freqs)
# ax.set_xlabel(r"Density (count/$\AA^2$)")
# ax.set_ylabel("Probability Density")
# plt.savefig("Distribution_of_densities.pdf")

In [ ]:
# enrichments = densities.copy()
# enrichments.iloc[0,:] =  enrichments.iloc[0,:]/0.118
# enrichments.iloc[1,:] =  enrichments.iloc[1,:]/0.0518
# fig, ax = polar_plot(enrichments, theta, radius, chains_groups, helices_lwr, helices_upr)
# plt.savefig("enrichments.pdf")
# plt.show()